<a href="https://colab.research.google.com/github/nihithabsk/Automatic_CO_PO_Attainment/blob/main/CO_PO_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd

file_path = "docs.xlsx"
sheet_name = 'DSP_2020_24_Asec_mid2'

def cal(file_path, sheet_name, class_strength):
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=0)

    def get_user_defined_co_mappings():
        co_mapping = {}
        while True:
            co_name = input("Enter CO name (e.g., 'Course Outcome 1') or type 'done' to finish: ").strip()
            if co_name.lower() == 'done':
                break
            questions = input(f"Enter the columns (comma-separated) for {co_name}: ").split(',')
            co_mapping[co_name] = [q.strip() for q in questions]
        return co_mapping

    co_mapping = get_user_defined_co_mappings()
    max_marks = df.iloc[0]
    df.columns = df.columns.str.strip()
    highest_marks = {}
    for co, questions in co_mapping.items():
        marks = 0
        for q in questions:
            if q in df.columns:
                max_mark = df.loc[df['Parameters'] == 'Max Marks', q]
                if not max_mark.empty:
                    marks += max_mark.values[0]
        highest_marks[co] = marks

    student_rows = df[df['Parameters'] != 'Max Marks'].copy()
    student_rows.dropna(axis=1, how='all', inplace=True)
    student_rows.reset_index(drop=True, inplace=True)

    for co, questions in co_mapping.items():
        for q in questions:
            student_rows[q] = pd.to_numeric(student_rows[q], errors='coerce')

    student_scores = pd.DataFrame(student_rows['Parameters'])
    for co, questions in co_mapping.items():
        student_scores[co] = student_rows[questions].sum(axis=1)

    student_scores.rename(columns={'Parameters': 'Student'}, inplace=True)

    renaming_map = {co: co.replace(' ', '_') for co in co_mapping.keys()}

    for co in co_mapping.keys():
        student_scores.rename(columns={co: renaming_map[co]}, inplace=True)
        student_scores.rename(columns={f'{co} Percentage': f'{renaming_map[co]}_Percentage'}, inplace=True)

    for co in co_mapping.keys():
        percentage_column = f'{renaming_map[co]}_Percentage'
        student_scores[percentage_column] = (student_scores[renaming_map[co]] / highest_marks[co]) * 100

    questions = [q for questions in co_mapping.values() for q in questions]
    student_question_scores = student_rows.copy()

    # Ensure 'Student' column is present before setting it as index
    if 'Student' not in student_question_scores.columns:
        student_question_scores['Student'] = student_rows['Parameters']
    student_question_scores.set_index('Student', inplace=True)

    question_columns = [q for questions in co_mapping.values() for q in questions]
    student_question_scores = student_question_scores[question_columns]
    student_question_scores = student_question_scores.iloc[:-17]

    stud_marks_in_each_ques = {col.strip(): student_question_scores[col].sum() for col in student_question_scores.columns}

    max_marks.index = max_marks.index.str.strip()
    marks_studA = {}
    marks_studB = {}
    for ques, sm in stud_marks_in_each_ques.items():
        ques = ques.strip()
        if ques in max_marks.index:
            marks_studA[ques] = max_marks[ques] * class_strength
            marks_studB[ques] = sm

    percentage_of_attainment = {}
    for ques in marks_studA.keys():
        percentage_of_attainment[ques] = (marks_studB[ques] / marks_studA[ques]) * 100

    avg_cos = {}
    for co in co_mapping.keys():
        sumb = 0
        suma = 0
        for q in co_mapping[co]:
            q = q.strip()
            sumb += marks_studB[q]
            suma += marks_studA[q]
        avg_cos[co] = (sumb / suma) * 100

    threshold_of_cos = {co: avg * (30 / 40) for co, avg in avg_cos.items()}

    student_scores = student_scores.iloc[:-17]

    students_above_threshold = {}
    for co, threshold in threshold_of_cos.items():
        percentage_column = f'{renaming_map[co]}_Percentage'
        if percentage_column in student_scores.columns:
            count_above_threshold = (student_scores[percentage_column] > threshold).sum()
            students_above_threshold[co] = count_above_threshold
        else:
            print(f"Percentage column for {co} not found in student_scores.")

    percentage_above_threshold = {}
    for co, count in students_above_threshold.items():
        percentage_above_threshold[co] = (count / class_strength) * 100

    avg = sum(percentage_above_threshold.values()) / len(percentage_above_threshold)

    if avg < 60:
        ans = "Level 0"
    elif 60 <= avg <= 70:
        ans = "Level 1"
    elif 70 < avg <= 80:
        ans = "Level 2"
    else:
        ans = "Level 3"

    return avg, ans

# Running the function
cal(file_path, sheet_name, 66)


Enter CO name (e.g., 'Course Outcome 1') or type 'done' to finish: Course Outcome 3
Enter the columns (comma-separated) for Course Outcome 3: 1a,1b,1c,1d,1e,1f,2a,2b,3a,3b,4a,4b,A1
Enter CO name (e.g., 'Course Outcome 1') or type 'done' to finish: Course Outcome 4
Enter the columns (comma-separated) for Course Outcome 4: 1g,1h,1i,1j,5a,5b,6a,A2
Enter CO name (e.g., 'Course Outcome 1') or type 'done' to finish: Course Outcome 5
Enter the columns (comma-separated) for Course Outcome 5: 6b,A3
Enter CO name (e.g., 'Course Outcome 1') or type 'done' to finish: done


(71.71717171717171, 'Level 2')